# Tmax States Final

A notebook to subset Tmax daily for the 13000 GHS urban areas to identify dates >40c, consecuritve days >40 c etc.

Moved from cpt_tmax_subset to clean up all the code on 2019-09-24

**Need to subset**
- Days per year 
- Duration of each event 
- Intensity of each day during each event (>40.6)
- Avg temp
- Avg intsensity

**NOTE 2019.10.19 FOUND BUG IN DATA ---> EVENTS TOO LONG**
Some events too long and have dates that are note sequential in them.

I think I fixed the problem on 2019.10.19 and reran the code


**NOTE 2019.10.19 RE RAN CODE TO PRODUCE STATS FOR ALL YEARS AFTER FIXING BUG**

#### Dependencies

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian

## Functions

#### #1 Function  turns csv into x-array

In [2]:
def csv_to_xr(file_in, time_dim, space_dim):
    
    """ Function reads in a csv w/ GHS-UCDB IDs and temp, isolates the temp
    and returns a xarray data array with dims set to city ids and dates
    
    Args:
        file_in = file name and path
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    
    df = pd.read_csv(file_in) # read the file in as a df
    print(df.shape)
    
    df_id = df[space_dim] # get IDs
    df_temp = df.iloc[:,3:] # get only temp columns
    df_temp.index = df_id # set index values
    df_temp_drop = df_temp.dropna() # Drop cities w/ no temp record 
    print(len(df_temp_drop))
    
    temp_np = df_temp_drop.to_numpy() # turn temp cols into an np array
    
    # make xr Data Array w/ data as temp and dims as spece (e.g. id)
    
    # Note 2019 09 17 changed to xr.Dataset from xr.Dataarray
    temp_xr_da = xr.DataArray(temp_np, coords=[df_temp_drop.index, df_temp_drop.columns], 
                            dims=[space_dim, time_dim])
    
    return temp_xr_da

#### #2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city

In [3]:
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    dayTot_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # this is actually getting the total events of all, 2019-09-22
        dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    df_out['total_days'] = dayTot_list
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

#### #3 Function splits the dataset into Tmax events (continuous days >Tmax) for each city

In [4]:
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax, total_days):
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        country: country for each city as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    tot_days = total_days
    
    # lists to fill
    city_id_list = []
    tot_days_list = []
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        
        # fill lists
        city_id_list.append(city_id)
        tot_days_list.append(tot_days)
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['total_days'] = tot_days_list
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### #4 Function feeds output from function 2 into function 3

In [5]:
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats
    
        NOTE - If you add arguments to event_split to make more states, 
        be sure to update this function
    
        args:
            df: input dataframe
        
    """
    df_out = pd.DataFrame()
    
    # NOTE - If you add arguments to event_split to make more stats, 
    # be sure to update this function
    
    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        total_days = row['total_days'] # get total number of tmax days

        df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)

        df_out = df_out.append(df)
    
    return df_out

#### #5 Loops through a file list and applies functions 1 - 4 to the data to produce Tmax stats for all tmax events in a given year across all cities in the dataset

In [6]:
def stats_loop(dir_in, dir_out, fn_out, time_dim, space_dim, Tthresh):
    
    """ Loop through a dir with csvs to apply csv_to_xr and
    tmax_stats function and save out a .csv for each year
    
    Args:
        dir_in = dir path to loop through
        dir_out = dir path to save files out
        fn_out = string to label out files
        time_dim = name for time dim as a str ... use date :-) for csv_to_xr function
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0) for csv_to_xr function
        Tthresh = int of temp threshold for temp_event function -- 40.6 is used

    """
    
    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
        
    # Git File list
    fn_list = glob.glob(dir_in+'*.csv')
    
    for fn in sorted(fn_list):
        
        # Get year for arg for temp_event function
        year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        print(year)
        
        # read csv as a data array
        temp_xr_da = csv_to_xr(fn, time_dim, space_dim)
        
        # data array to tmax events, out as df
        df_days = tmax_days(temp_xr_da, Tthresh)
        
        # tmax events stats, out as df
        df_out = tmax_stats(df_days)
        
        # merge to get countries
        ghs_ids_df_out = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'inner') 
        
        # write it all out
        ghs_ids_df_out.to_csv(dir_out+fn_out+year+'.csv')

        print(year, 'SAVED!')

#### Run Code

In [2]:
dir_in = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/' # ghs ID list
dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
fn_out = 'CHIRTS-GHS-Events-Stats'
time_dim = 'date'
space_dim = 'ID_HDC_G0'
Tthresh = 40.6


In [8]:
#stats_loop(dir_in, dir_out, fn_out, time_dim, space_dim, Tthresh)

1983
(13135, 368)
13067
1983 SAVED!
1984
(13135, 369)
13067
1984 SAVED!
1985
(13135, 368)
13067
1985 SAVED!
1986
(13135, 368)
13067
1986 SAVED!
1987
(13135, 368)
13067
1987 SAVED!
1988
(13135, 369)
13067
1988 SAVED!
1989
(13135, 368)
13067
1989 SAVED!
1990
(13135, 368)
13067
1990 SAVED!
1991
(13135, 368)
13067
1991 SAVED!
1992
(13135, 369)
13067
1992 SAVED!
1993
(13135, 368)
13067
1993 SAVED!
1994
(13135, 368)
13067
1994 SAVED!
1995
(13135, 368)
13067
1995 SAVED!
1996
(13135, 369)
13067
1996 SAVED!
1997
(13135, 368)
13067
1997 SAVED!
1998
(13135, 368)
13067
1998 SAVED!
1999
(13135, 368)
13067
1999 SAVED!
2000
(13135, 369)
13067
2000 SAVED!
2001
(13135, 368)
13067
2001 SAVED!
2002
(13135, 368)
13067
2002 SAVED!
2003
(13135, 368)
13067
2003 SAVED!
2004
(13135, 369)
13067
2004 SAVED!
2005
(13135, 368)
13067
2005 SAVED!
2006
(13135, 368)
13067
2006 SAVED!
2007
(13135, 368)
13067
2007 SAVED!
2008
(13135, 369)
13067
2008 SAVED!
2009
(13135, 368)
13067
2009 SAVED!
2010
(13135, 368)
13067
2010

## QA/QC

In [4]:
# Load files

dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
fn_out = 'CHIRTS-GHS-Events-Stats'

In [6]:
qc_df = pd.read_csv(dir_out+fn_out+'1983.csv')

In [7]:
qc_df.head()

,Unnamed: 0,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax
0,0,2784,Germany,1,1,44.459750,3.859750,3.859750,['1983.07.27'],[3.85975],[44.45975]
1,1,2372,France,1,1,43.331635,2.731635,2.731635,['1983.07.31'],[2.731635],[43.331635]
2,2,6156,Kazakhstan,1,1,41.336376,0.736376,0.736376,['1983.07.31'],[0.736376],[41.336376]
3,3,2833,Germany,1,1,47.493180,6.893180,6.893180,['1983.07.27'],[6.89318],[47.49318]
4,4,2885,Austria,1,1,44.013890,3.413890,3.413890,['1983.07.27'],[3.41389],[44.01389]


In [8]:
dc_city = qc_df[qc_df['ID_HDC_G0'] == 5534]

In [9]:
dc_city

,Unnamed: 0,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax
22525,22525,5534,Ethiopia,84,1,40.736885,0.136885,0.136885,['1983.01.05'],[0.136885],[40.736885]
22526,22526,5534,Ethiopia,84,13,41.737855,1.137855,14.792109,['1983.01.19' '1983.01.20' '1983.01.21' '1983....,[0.846815 2.002577 1.19209 0.916884 0.81514 ...,[41.446815 42.602577 41.79209 41.516884 41.41...
22527,22527,5534,Ethiopia,84,9,41.062670,0.462670,4.164027,['1983.02.05' '1983.02.06' '1983.02.07' '1983....,[0.137976 0.035193 0.511267 0.594824 1.123145 ...,[40.737976 40.635193 41.111267 41.194824 41.72...
22528,22528,5534,Ethiopia,84,8,41.553627,0.953627,7.629014,['1983.02.16' '1983.02.17' '1983.02.18' '1983....,[0.600165 1.43894 1.109015 1.421606 1.34052 ...,[41.200165 42.03894 41.709015 42.021606 41.94...
22529,22529,5534,Ethiopia,84,43,42.115429,1.515429,65.163434,['1983.02.26' '1983.02.27' '1983.02.28' '1983....,[0.33683 1.354803 1.337897 1.908553 0.734724 ...,[40.93683 41.954803 41.937897 42.508553 41.33...
22530,22530,5534,Ethiopia,84,10,41.332326,0.732326,7.323264,['1983.04.11' '1983.04.12' '1983.04.13' '1983....,[0.224825 0.785494 0.962283 1.486514 0.874148 ...,[40.824825 41.385494 41.562283 42.086514 41.47...


## Testing

In [ ]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/data_out_urbanheat/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out/'

In [ ]:
# File name to test
fn_in = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
# Open a raw file
xr1983 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

In [ ]:
days1983 = tmax_days(xr1983, 40.6)

In [ ]:
test = days1983[0:30]
test

# Maybe add in days_total

In [ ]:
# Build routine for loop through a csv

df_out = pd.DataFrame()

for index, row in test.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days
    
    df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)
    
    df_out = df_out.append(df)

In [ ]:
df_out

# Fixing code
2019.10.19 Cascade Tuholkse

Need to fix ```event split``` function

Somewhere in 1984 is this event sequence: ['1984.01.01' '1984.01.02' '1984.01.07']


**FOUND PROBLEM AND IT IS FIXED**

In [ ]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/data_out_urbanheatv/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out_urbanheat/testout/'

In [ ]:
# File name to test
fn_in = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
# Open a raw file
xr1983 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

In [ ]:
# Find all the tmax days
tmax1983 = tmax_days(xr1983, 40.6)

In [ ]:
# Make a subset

test = tmax1983[tmax1983['ID_HDC_G0'] == 6279]
test

In [ ]:
for index, row in test.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days



In [ ]:
# Convert to jul days
jul_days = pd.to_datetime(dates).to_julian_date()
jul_days


In [ ]:
mjd = 2445507.5 - 43200
dt = julian.from_jd(mjd, fmt='mjd')
print(dt)


In [ ]:
dates

In [ ]:
dates = ['1983.06.20', '1983.06.23', '1983.06.24', '1983.06.25',
        '1983.06.26', '1983.06.27', '1983.06.28', '1983.06.29',
        '1983.06.30', '1983.07.01', '1983.07.21', '1983.07.22',
        '1983.07.23', '1983.08.01']

pd_dates = pd.to_datetime(dates)
df_dates = pd.DataFrame()
df_dates['dates'] = pd_dates



test = df_dates['dates'].apply(lambda x: x.toordinal())

In [ ]:
from datetime import date
date.fromordinal(test[1])

In [ ]:
counter = 0

In [ ]:
event_dates = dates[0:10] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

event_dates = dates[10:13] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

event_dates = dates[13:14] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

In [ ]:
event_dates_list = []
counter = 0
start = 0
end = 0

for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
    
    len_dates = len(dates) # len of all Tmax dates for a given city
#   print(len(dates))
    
    seq = list(map(itemgetter(1), g)) # isolate seq. days
    dur = len(seq) # duration of each event 
    
    counter = counter + dur
    end = counter
    
    print(end)
    
    event_dates = dates[start:end] # dates of tmax days during each event
    print(event_dates)
    
    start = counter
    
# #     print('dur= ', dur)
    
#     event_dates = dates[0:10] # dates of tmax days during each event
#     print(event_dates)
    
#     event_dates = dates[11:13] # dates of tmax days during each event
#     print(event_dates)
    
#     event_dates = dates[:len_dates] # dates of tmax days during each event
#     print(event_dates)
    
    
    
    
#     counter = counter + dur

#     print('counter = ', counter)
    
#     dif = dur - counter
#     print('dif = ', dif)
    
#     start = start 
#     print('start = ',start)
    
# #     end = counter + dur
# #     print("start = ",end)
    
    
    
#     counter = counter + dur
#     print(counter)
#     end = counter + dur
#     start
#     event_dates = dates[start:end] # dates of tmax days during each event
#     print(event_dates)
# #     intense = intensity[0:dur] # intensity of each day during event
#     temp = tmax[0:dur] # temp of each day during event
#     avg_temp = mean(temp) # avg. temp during event
#     avg_int = mean(intense) # avg. intensity during event
#     tot_int = intense.sum() # total intensity during event
    
#     event_dates_list.append(event_dates)
    


In [ ]:
event_dates_list

In [ ]:
df = pd.DataFrame()
df['event_dates'] = event_dates_list
df

In [ ]:
import more_itertools as mit

In [ ]:
# run tmax stats ------> CLEARLY NOTE WORKING

# tmax1983_sub_stats = tmax_stats(tmax1983_sub)

In [ ]:
type(tmax1983['dates'][1])

In [ ]:
dates = tmax1983_sub['dates']
#jul_days = pd.to_datetime(tmax1983['dates']).to_julian_date()

In [ ]:
# Try to break up events

for index, row in tmax1983_sub.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days

#     df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)

#     df_out = df_out.append(df)

dates

In [ ]:
type(df_out['events'][0])